In [56]:
# Imports
import json
import numpy as np
import pandas as pd

# Load in the data file
with open('../question-data.json') as data_file:
    data = json.loads(data_file.read())

# Prepare the data
td_text = []
td_labels = []
for index in data:
    td_text += [data[index][x]for x in range(len(data[index]))]
    td_labels += [index for x in range(len(data[index]))]
#final = {"text": td_text, "label": td_labels}
str_to_num = {'overview': 0, 'injury': 1, 'trade': 2}
final = [{"text": td_text[i], "label": str_to_num[td_labels[i]], "idx": i} for i in range(len(td_text))]

In [57]:
# Writing to a json file
import json
json_object = json.dumps(final, indent=4)
with open("final_data.json", "w") as outfile:
    outfile.write(json_object)

In [58]:
# Load the dataset
from datasets import load_dataset
dataset = load_dataset('json', data_files='final_data.json') #, split=['train[:80%]', 'train[20%:]'])

Using custom data configuration default-57a84150c0601514


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /Users/danielstefanescu/.cache/huggingface/datasets/json/default-57a84150c0601514/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [59]:
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

  0%|          | 0/1 [00:00<?, ?ba/s]

In [60]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

# tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset(
#     columns=["attention_mask", "input_ids", "token_type_ids"],
#     label_cols=["labels"],
#     shuffle=False,
#     collate_fn=data_collator,
#     batch_size=8,
# )

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [61]:
# Training the model

from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam

# Load and compile our model
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", id2label={0: 'overview', 1: 'injury', 2: 'trade'})
# Lower learning rates are often better for fine-tuning transformers
model.compile(optimizer=Adam(3e-5))

model.fit(tf_train_dataset)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.
2022-10-19 12:49:11.772880: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 39s 8s/step - loss: 1.2037


In [67]:
from transformers import TextClassificationPipeline
clf = TextClassificationPipeline(model=model, tokenizer=tokenizer)

In [70]:
labels = [111, 116, 105]
print (clf("When did [player] begin their professional career?"))

[{'label': 'overview', 'score': 0.5830708146095276}]
